# PgSTAC Tutorial
This tutorial is designed to run in the docker compose environment defined in the docker-compose.yml file at the root of the pgstac git repository. For instructions on installing Docker and Docker Compose, you can go to https://docs.docker.com/get-docker/. The instructions provided here use the newer Compose V2 `docker compose` rather than `docker-compose` now that Docker Compose is included as part of latest version of Docker.

To get started with this tutorial you should already have checked out the pgstac repository from github using `git clone https://github.com/stac-utils/pgstac`

To start the database, a STAC FastAPI instance, and the Jupyter Notebook for this tutorial: 
```
cd docs/src/tutorial
docker compose up -d tutorial
```

- You can then start up the workshop by going to http://localhost:8891/notebooks/tutorial.ipynb?token=token
- You can get to a STAC FastAPI PgSTAC instance at http://localhost:8890 (although this will not work until we have installed PgSTAC onto our Database later)


In [9]:
!docker run --name pgstac-tutorial -e POSTGRES_PASSWORD=password -e PGPASSWORD=password -d postgis/postgis


docker: Error response from daemon: Conflict. The container name "/pgstac-tutorial" is already in use by container "5bf96cc3a75e1ec90e65c42a4e0da356a728d46a8e746731c0af457713533ea2". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [6]:
!docker exec pgstac-tutorial psql -l -U postgres


                                                    List of databases
       Name       |  Owner   | Encoding |  Collate   |   Ctype    | ICU Locale | Locale Provider |   Access privileges   
------------------+----------+----------+------------+------------+------------+-----------------+-----------------------
 postgres         | postgres | UTF8     | en_US.utf8 | en_US.utf8 |            | libc            | 
 template0        | postgres | UTF8     | en_US.utf8 | en_US.utf8 |            | libc            | =c/postgres          +
                  |          |          |            |            |            |                 | postgres=CTc/postgres
 template1        | postgres | UTF8     | en_US.utf8 | en_US.utf8 |            | libc            | =c/postgres          +
                  |          |          |            |            |            |                 | postgres=CTc/postgres
 template_postgis | postgres | UTF8     | en_US.utf8 | en_US.utf8 |            | libc            | 


## IPython Magic

This installs a "magic" command for the Jupyter Notebook that will allow us to run psql commands to help us explore the database.
The `%psql` line magic is the same as running a command using psql in the terminal.
The `%%psql` cell magic runs the rest of the cell as the stdin to psql. This tutorial uses a system call to the psql utility. This is already installed on the Docker image that comes with this Tutorial. On Debian/Ubuntu clients, this can be installed using `sudo apt install Postgres-client`.

In [2]:
from IPython.core.magic import register_line_cell_magic
from IPython.display import display, HTML, JSON
import orjson
import shlex
from subprocess import Popen, PIPE

@register_line_cell_magic
def psql(line=None, cell=None):
    args = ["-X"] + (shlex.split(line) or ["-H"])
    if '-1' in args:
        args += ['-v', 'ON_ERROR_STOP=1']
    if cell:
        args += ['-f', '/dev/stdin']
    else:
        cell = ''
    r=Popen(['psql', *args], stdin=PIPE, stdout=PIPE, stderr=PIPE, text=True)
    out, err = r.communicate(input=cell)

    if "-H" in args:
        print(err)
        display(HTML(out))
    elif "-At":
        print(err)
        for line in out.strip().split('\n'):
            try:
                display(orjson.loads(line))
            except:
                display(line)
    else:
        print(err)
        print(out)


## Check the standard Postgres environment variables 
Most tools that work with Postgres use the standard environment variables that are used by all of the tools that come standard as part of Postgres. The pypgstac python utility that comes with pgstac and is installable from pip.

In [3]:
%env PGDATABASE=pgstac
%env PGUSER=adminrole
%env PGPASSWORD=password
%env PGHOST=localhost
%env PGPORT=5438


env: PGDATABASE=pgstac
env: PGUSER=adminrole
env: PGPASSWORD=password
env: PGHOST=localhost
env: PGPORT=5438


Check that we can login to the database. We will use a call out to the command line psql utility using the `-l` option to list all databases.

In [4]:
%psql -H -l


psql: error: connection to server at "localhost" (127.0.0.1), port 5438 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?



## Install PgSTAC to the Database
This docker-compose.yml with this tutorial uses the postgis/postgis:15-3.4 docker image as the base Postgres. PgSTAC requires Postgres>=14 and PostGIS>=3.1. We will now install PgSTAC on the database using the command line `pypgstac migrate` tool. Pypgstac is already installed in the docker image running this notebook. To install otherwise `pip install --upgrade pypgstac[psycopg]`.

In [4]:
!pypgstac migrate 0.8.1 --debug


DEBUG:pypgstac.db:PG VERSION: 15.4 (Debian 15.4-2.pgdg110+1).
INFO:pypgstac.migrate:Migrating PGStac on PostgreSQL Version 15.4 (Debian 15.4-2.pgdg110+1)
DEBUG:pypgstac.db:VERSION: 0.8.1
INFO:pypgstac.migrate:Target database already at version: 0.8.1
0.8.1


## Check PgSTAC Install
We can use the psql command line utility to login to our database now and to look around. Let's show what schemas are in the database. SQL Commands beginning with `\` are meta commands in psql. In this case `\dn` is asking to show all schemas (also called namespaces which explains the "n"). We can see that we have a "public" schema which is there by default in all Postgres instances as well as a "pgstac" schema that is owned by the "pgstac_admin" role - this schema as well as the pgstac_admin role were created by the `pypgstac migrate` tool.

In [5]:
%%psql
\dn


Name,Owner
pgstac,pgstac_admin
public,pg_database_owner
tiger,adminrole
tiger_data,adminrole
topology,adminrole


Postgres, much like your shell or python, has a configurable path that it uses to find anything in the database. It is controlled by the "SEARCH_PATH" [setting](https://www.Postgres.org/docs/current/config-setting.html) in Postgres. Postgres will search each schema in the order defined by the "SEARCH_PATH" to find database objects (tables, functions, views, etc). By default, the "SEARCH_PATH" is set to search a schema with the same name as the currently logged in role (which is "pgstac" with the docker environment we are using) followed by the "public" schema.

In [7]:
%%psql
SHOW SEARCH_PATH;


search_path
"pgstac, public"


Since we have installed PgSTAC into the "pgstac" schema, we need to make sure that "pgstac" is available in our envrionment. We can do this temporarily using the "SET" command in Postgres `SET SEARCH_PATH TO pgstac, public;`. Or, we can modify the setting at the DATBASE or ROLE level. For this tutorial, we will set the default setting for "SEARCH_PATH" at the DATABASE level.

In [8]:
%%psql
ALTER DATABASE pgstac SET SEARCH_PATH TO pgstac,public;
SHOW SEARCH_PATH;


search_path
"pgstac, public"


## Configuring Postgres Specific Settings
There are many other settings that can be set at the SYSTEM, DATABASE, ROLE, or SESSION level. If something is set at multiple levels, the most specific level would win, so even though we have set the SEARCH_PATH at the DATABASE level, we could override it in a SESSION by using `SET SEARCH_PATH TO ...`.

Out of the box as well as on most hosted services, the default Postgres configuration is extremely conservative and should be adjusted. There is *NO* one-size-fits-all set of settings even for a given database host instance size. PgSTAC comes with a function that can help to determine what a good starting point may be for some of the most important settings. Fine tuning a database can be an entire career though, so it is important to undestand some of the factors where you may want to adjust these settings. The function takes a single argument which is the memory size of the instance.

In [9]:
%%psql
SELECT check_pgstac_settings('16GB');


psql:/dev/stdin:1: NOTICE:  effective_cache_size of 12 GB is set appropriately for a system with 16 GB
psql:/dev/stdin:1: NOTICE:  shared_buffers of 4096 MB is set appropriately for a system with 16 GB
psql:/dev/stdin:1: WARNING:  maintenance_work_mem of 512 MB is set low for shared_buffers of 4915 MB. Recomended value between 983 MB and 1475 MB
psql:/dev/stdin:1: WARNING:  work_mem setting of 128 MB is set low for 20 max_connections you may consider raising work_mem to 246 MB or increasing max_connections to 38
psql:/dev/stdin:1: NOTICE:  random_page_cost and seq_page_cost set appropriately for SSD
psql:/dev/stdin:1: NOTICE:  VALUES FOR PGSTAC VARIABLES
psql:/dev/stdin:1: NOTICE:  These can be set either as GUC system variables or by setting in the pgstac_settings table.
psql:/dev/stdin:1: NOTICE:  context is set to off from the pgstac_settings table
psql:/dev/stdin:1: NOTICE:  context_estimated_count is set to 100000 from the pgstac_settings table
psql:/dev/stdin:1: NOTICE:  context_

check_pgstac_settings
""


### Important Settings That You Should Always Review

#### [effective_cache_size](http://www.Postgres.org/docs/current/static/runtime-config-resource.html#GUC-SHARED-BUFFERS)
This is the amount of memory that is estimated to be left on the system for the OS and all other processes. This is generally 1/2 to 3/4 of the total system memory.

#### [shared_buffers](http://www.Postgres.org/docs/current/static/runtime-config-resource.html#GUC-SHARED-BUFFERS)
This setting is used to tell the database how much memory is available to dedicate to Postgres for caching data. General rule-of-thumb is to set this to 1/4 of the total system memory.

#### [work_mem](http://www.Postgres.org/docs/current/static/runtime-config-resource.html#GUC-WORK-MEM) and [max_connections](http://www.Postgres.org/docs/current/static/runtime-config-connection.html#GUC-MAX-CONNECTIONS)
This is the memory that is allowed to be used per sort operation per connection for things like sorting and complex queries. This setting will really vary with the use of the database and the number of max connections that are needed in the database. In general `max_connections * work_mem` should be less than the setting for `shared_buffers`. If you have individual queries that you know will be doing larger sorts, the `work_mem` setting can be set at run time: `SET work_mem TO '40MB';`

#### [maintenance_work_mem](http://www.Postgres.org/docs/current/static/runtime-config-resource.html#GUC-MAINTENANCE-WORK-MEM)
This is the amount of memory that is made available for operations such as Vacuuming the database and Creating Indexes. This memory will only be used once at a given time, so it is OK to set this significantly higher than work_mem. 1/4 of the `shared_buffers` is a reasonable place to set this.

#### [seq_page_cost](http://www.Postgres.org/docs/current/static/runtime-config-query.html#GUC-SEQUENTIAL-PAGE-COST) and [random_page_cost](https://www.Postgres.org/docs/current/runtime-config-query.html#GUC-RANDOM-PAGE-COST)
These two variables are interpreted together and it is the ratio of `random_page_cost / seq_page_cost` that really matters. Generally, `seq_page_cost` should be left at the default of 1 and random_page_cost should be changed to reflect the nature of the underlying storage. The default for `random_page_cost` is set to 4 which is appropriate for Spinning Hard Disk Drives. For use with Solid State Drives (which is almost always what a modern hosted platform such as RDS uses), this should be set to 1.1. Having `random_page_cost` set too high can lead to wayyyy slower queries as the Postgres query planner will tend to prefer sequential scans over index scans for many queries.

#### [temp_buffers](https://www.Postgres.org/docs/current/runtime-config-resource.html#GUC_TEMP_BUFFERS)
If using Temporary Tables, increasing this setting can help to avoid spilling to disk. 

In [10]:
%%psql
ALTER SYSTEM SET EFFECTIVE_CACHE_SIZE TO '12GB';
ALTER SYSTEM SET SHARED_BUFFERS TO '4GB';
ALTER SYSTEM SET WORK_MEM TO '128MB';
ALTER SYSTEM SET MAINTENANCE_WORK_MEM TO '512MB';
ALTER SYSTEM SET MAX_CONNECTIONS TO 20;
ALTER SYSTEM SET RANDOM_PAGE_COST TO 1.1;
ALTER SYSTEM SET TEMP_BUFFERS TO '512MB';
SELECT pg_reload_conf();


pg_reload_conf
t


Note that these settings could also be set in the postgres.conf settings file on the database server or using the configuration editing tools provided by most Database as a Service providers. 

## PgSTAC Roles
PgSTAC installs three roles with different limitations.
- "pgstac_admin" is the owner of the pgstac schema and all items in the schema. This role has the ability to use or modify anything in pgstac and should be used sparingly.
- "pgstac_ingest" has read/write permissions to create items and collections in PgSTAC, but not to modify any of the PgSTAC utilities. This role should be used when you need access to ingest or modify data in the PgSTAC Catalog.
- "pgstac_read" is the primary role that should be used when accessing PgSTAC when not writing any data. It is not, however, a strictly read-only role as there are still cache and statistics tables which the role will write to behind the scenes.

### Assuming a role
The role that we are using so far is an administrative or root user of the database. While you need to use a role with sufficient priviliges to create a schema, you should never use this role when accessing Postgres for working with PgSTAC. The PgSTAC roles are not set up by default to be able to login to the database, but we can use Role Inheritance to be able to assign another role with all the privileges of one of hte PgSTAC roles.

Best practice would be to create a role that is used for ingest or transactional tasks and one that is used when just reading STAC Items and Collections.

In [11]:
%%psql -U adminrole
CREATE ROLE stacrw WITH LOGIN PASSWORD 'password' IN ROLE pgstac_ingest;
CREATE ROLE stacr WITH LOGIN PASSWORD 'password' IN ROLE pgstac_read;


psql:/dev/stdin:1: ERROR:  role "stacrw" already exists
psql:/dev/stdin:2: ERROR:  role "stacr" already exists




For the rest of the tutorial, we will not be needing to perform any administrative tasks, so we can change our environment variables so that we login as the "stacrw" role.

In [12]:
%env PGUSER=stacrw
%env PGPASSWORD=password


env: PGUSER=stacrw
env: PGPASSWORD=password


Let's double check our SEARCH_PATH.

In [13]:
%%psql
SELECT current_user;
SHOW SEARCH_PATH;


current_user
stacrw
search_path
"pgstac, public"


## PgSTAC Data Layout
PgSTAC does not directly store STAC Items and Collections as JSON. Rather it pulls some of the information out into properly typed separate columns that can more effectively be used for searching through STAC Items. This data layout is intended to be a back-end implementation and particularly for the "items" table these tables should not be used directly for SELECT/INSERT/DELETE.

### Items Table
If we look a little closer at the "items" table, we can see that it is actually a parent partition. No data is actually stored directly in the items table, but rather in children partitions that are created through the use of triggers on the "collections" table. Right now, you can see that we have a foreign key constraint on the "collections" table (so, you must have a collection added to PgSTAC before adding any "items"). As of now, there are no partitions as we have not added any data yet.

#### Items Table Layout
- id: This is the id from the original JSON Item
- geometry: The geojson from the original JSON item has been extracted and saved as a PostGIS Geometry column.
- collection: The Collection id which is set as a Foreign Key Constraint
- datetime: If the Item JSON has properties.datetime set, this comes from that, otherwise it comes from properties.start_datetime
- end_datetime: If the Item JSON has properties.datetime set, this comes from that, otherwise it comes from properties.end_datetime
- content: This is the remainder of the original JSON Item after removing the geometry, id, and collection as well as well as using a form of compression based on the common item_assets stored with a collection. This is discussed further under "Hydration".
- private: This field is currently not used directly by PgSTAC, but it is to provide a place where additional private metadata about an item that is not part of the public STAC record (ie access constraints, etc) could be stored.

Note that we always have a date range that we can use between datetime and end_datetime (where in the case of a "datetime" in the original JSON represents an instant in time.

In [14]:
%%psql
\d+ items


Column,Type,Collation,Nullable,Default,Storage,Compression,Stats target,Description
id,text,,not null,,extended,,,
geometry,geometry,,not null,,main,,,
collection,text,,not null,,extended,,,
datetime,timestamp with time zone,,not null,,plain,,,
end_datetime,timestamp with time zone,,not null,,plain,,,
content,jsonb,,not null,,extended,,,
private,jsonb,,,,extended,,,


## Collections Table
You'll notice that the "collections" table is layed out fairly similary to the Items table. These common columns help enable using the same tooling for search for both Items and Collections. As the "collections" table is generally much much smaller than the "items" table, there are fewer concerns for scalability and this table is not partitioned as is the "items" table.

### Collections Table Layout
- key: This is an integer primary key that is generated upon creation
- id: This is the id from the original JSON Collection
- geometry: The total bounds from the Collection extent.spatial_extent has been extracted and saved as a PostGIS Geometry column.
- datetime: The start of the Collection extent.temporal_extent
- end_datetime: The end of the Collection extent.temporal_extent
- content: The full original Collection JSON
- base_item: This is used internally for "Hydration" process used to help compress Item records
- private: This field is currently not used directly by PgSTAC, but it is to provide a place where additional private metadata about an item that is not part of the public STAC record (ie access constraints, etc) could be stored.
- partition_trunc: This is used to control how finely partitioned the Items for a Collection are in the "items" table.

In [15]:
%%psql
\d collections


Column,Type,Collation,Nullable,Default
key,bigint,,not null,generated always as identity
id,text,,,generated always as (content ->> 'id'::text) stored
content,jsonb,,not null,
base_item,jsonb,,,generated always as (collection_base_item(content)) stored
geometry,geometry,,,generated always as (collection_geom(content)) stored
datetime,timestamp with time zone,,,generated always as (collection_datetime(content)) stored
end_datetime,timestamp with time zone,,,generated always as (collection_enddatetime(content)) stored
private,jsonb,,,
partition_trunc,text,,,


## Loading Data
A number of things must happen when we load data into PgSTAC
- All data must be transformed into the table layout for Items and Collections
- An Items Collection must be loaded prior to any Items for that Collection
 - When adding a collection, it should be configured for the appropriate partitioning strategy.
- For Items, PgSTAC must create any partitions that are needed prior to actually loading the data in place.
- With partitioned data, Postgres allows for the creation of constraints that allow the Postgres query planner to skip partitions based on query predicates. When loading data, if new data must be checked and the constraints must be modified if necessary.

The management of partitions and constraints can take out fairly aggressive locks on the database when done within the same transaction as loading large amounts of data. This adds a lot more overhead to any transaction loading data into the database than normal Postgres or database loads. When planning how to load data into PgSTAC, it can be very beneficial to try to group inserts into larger chunks of data (but not too large, ~10,000 seems to be a happy medium). PgSTAC should be able to reasonably handle concurrent data loads, but as the locks taken out when modifying partitions/constraints are generally at the partition level, it is good to try to avoid concurrent writes of data into the same partitions.

### Pypgstac Loader
To help mitigate some of the issues with data loading, pypgstac comes with a tool to help bulk load data into PgSTAC. When creating pipelines or doing bulk loading of data into PgSTAC, this should be the preferred method. Data can be loaded via the STAC API when exposed by STAC FastAPI, but when loading via an exposed API, the entire load is done as a single transaction and it also requires a double network hop for all the data (Client -> FastAPI -> PgSTAC).

#### Load Collection

In [16]:
!pypgstac load collections collections.ndjson.gz --debug --method ignore


DEBUG:pypgstac.db:VERSION: 0.8.1
INFO:pypgstac.db:NOTICE - table "tmp_collections" does not exist, skipping
DEBUG:smart_open.smart_open_lib:{'uri': 'collections.ndjson.gz', 'mode': 'r', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
DEBUG:smart_open.smart_open_lib:encoding_wrapper: {'fileobj': <gzip _io.BufferedReader name='collections.ndjson.gz' 0x7f155e57add0>, 'mode': 'r', 'encoding': 'UTF-8', 'errors': None, 'newline': None}
DEBUG:pypgstac.load:INSERT 0 0
DEBUG:pypgstac.load:Rows affected: 0


##### DELETE THIS

In [1]:
%%psql -U adminrole
/*
SET ROLE pgstac_admin;
SET SEARCH_PATH TO pgstac, public;
\i /home/bitner/data/pgstac/src/pgstac/sql/003b_partitions.sql

GRANT ALL ON ALL TABLES IN SCHEMA pgstac to pgstac_ingest;
GRANT USAGE ON ALL SEQUENCES IN SCHEMA pgstac to pgstac_ingest;
\df+ update_partition_stats_q
*/


UsageError: Cell magic `%%psql` not found.


#### Load Items

In [18]:
%%psql
SELECT current_user;


current_user
stacrw


In [19]:
%%psql -U adminrole
GRANT ALL ON ALL TABLES IN SCHEMA pgstac to pgstac_ingest;



GRANT



In [20]:
%env PGUSER=stacrw
!pypgstac load items items.ndjson.gz --debug --method upsert


env: PGUSER=stacrw


DEBUG:pypgstac.db:VERSION: 0.8.1
DEBUG:smart_open.smart_open_lib:{'uri': 'items.ndjson.gz', 'mode': 'r', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
DEBUG:smart_open.smart_open_lib:encoding_wrapper: {'fileobj': <gzip _io.BufferedReader name='items.ndjson.gz' 0x7f19b7773940>, 'mode': 'r', 'encoding': 'UTF-8', 'errors': None, 'newline': None}
DEBUG:pypgstac.load:Found modis-13Q1-061 with base_item {'type': 'Feature', 'assets': {'hdf': {'type': 'application/x-hdf', 'roles': ['data'], 'title': 'Source data containing all bands'}, 'metadata': {'type': 'application/xml', 'roles': ['metadata'], 'title': 'Federal Geographic Data Committee (FGDC) Metadata'}, '250m_16_days_EVI': {'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': '16 day EVI', 'raster:bands': [{'unit': 'EVI', 'scale': 0.0001, 'data_type': 'int16', 'spatial_resoluti

#### Partition Strategy
By default, all Items are partitioned by Collection. 

In [127]:
%%psql
\d+ items
SELECT * FROM partitions WHERE collection='modis-13Q1-061';


This now shows that there is a partition "_items_1" in the "items" table that contains all data for the 'modis-13Q1-061' data. PgSTAC can also be configured per Collection to further sub-partition data using the datetime by year or month. This is controlled by the "partition_trunc" column on the Collections table. There is a trigger on the Collections table that will make sure that all data is repartitioned on any change of that column. It should be noted, that it is FAR BETTER to modify the partition_trunc value before loading any data. Any time we change that value it requires a full lock on that Collection and a full rewrite of all that data.

Let's change the strategy for the collection we just loaded...

In [21]:
%%psql
SELECT
    collection,
    CASE
        WHEN 'month' IS NULL THEN '-infinity'::timestamptz
        ELSE date_trunc('month', datetime)
    END as d,
    tstzrange(min(datetime),max(datetime),'[]') as dtrange,
    tstzrange(min(end_datetime),max(end_datetime),'[]') as edtrange
FROM _items_1
GROUP BY 1,2;


collection,d,dtrange,edtrange
modis-13Q1-061,2023-09-01 00:00:00+00,"[""2023-09-06 00:00:00+00"",""2023-09-30 00:00:00+00""]","[""2023-09-21 23:59:59+00"",""2023-10-15 23:59:59+00""]"
modis-13Q1-061,2023-08-01 00:00:00+00,"[""2023-08-29 00:00:00+00"",""2023-08-29 00:00:00+00""]","[""2023-09-13 23:59:59+00"",""2023-09-13 23:59:59+00""]"


In [22]:
%%psql
UPDATE collections SET partition_trunc='year';
\d+ _items_1
SELECT * FROM partitions WHERE collection='modis-13Q1-061';


psql:/dev/stdin:1: NOTICE:  Collection Trigger. modis-13Q1-061 1




- PgSTAC settings
  - How to set
    - GUC
    - pgstac_settings
    - search conf object
  - context
    - context_estimated_count
    - context_estimated_cost
    - context_stats_ttl
  - additional_properties
  - use_queue
  - queue_timeout
  - update_collection_extent
- PgSTAC Roles
  - pgstac_admin
  - pgstac_ingest
  - pgstac_read
- Database Structure
  - Partitions
  - Triggers
- Data Ingest
  - pypgstac loader
  - magic tables
  - functions
- Loading Data
  - Collection Required
  - Primary Key is id by partition
  - Considerations of transactions, concurrency
    - concurrency issues mostly when loading to same partition
  - Partition Trunc Strategy
    - Changing Partitions
  - Query Queue
  - Update Collection Extent
  - Hydration
- Partition Metadata
- Search
  - Chunked Search for Order by Datetime
  - Context
  - Queryables
    - additional_properties
    - wrappers
    - indexing
      - warning about too many indexes
  - Filter / CQL2-JSON
- STAC FastAPI
- pgstac.rs etc 

In [23]:
from pypgstac.db import PgstacDB
with PgstacDB(debug=True) as db:
    print(db.search())


INFO:pypgstac.db:NOTICE - FILTER: <NULL>
INFO:pypgstac.db:NOTICE - SEARCH:TOKEN: <NULL>
INFO:pypgstac.db:NOTICE - ***TOKEN: <NULL>
INFO:pypgstac.db:NOTICE - FULL WHERE CLAUSE:  TRUE 
INFO:pypgstac.db:NOTICE - Time to get counts and build query 22.296
INFO:pypgstac.db:NOTICE - Getting hydrated data.
INFO:pypgstac.db:NOTICE - CACHE SET TO f
INFO:pypgstac.db:NOTICE - Time to set hydration/formatting 0.51
INFO:pypgstac.db:NOTICE - Getting chunks for  TRUE  datetime DESC, id DESC
INFO:pypgstac.db:NOTICE - Running Query for 2023-01-01 00:00:00+00 to 2024-02-01 00:00:00+00. 2.844
INFO:pypgstac.db:NOTICE - Returned 11/11 Rows From 2023-01-01 00:00:00+00 to 2024-02-01 00:00:00+00. 0 to go. Time: 0.996ms
INFO:pypgstac.db:NOTICE - SEARCH_ROWS TOOK 4.182ms
INFO:pypgstac.db:NOTICE - Time to fetch rows 70.472
INFO:pypgstac.db:NOTICE - Query returned 11 records.
INFO:pypgstac.db:NOTICE - HAS NEXT | modis-13Q1-061:MOD13Q1.A2023273.h33v09.061.2023290142939
INFO:pypgstac.db:NOTICE - Time to get prev/nex

{"next":"modis-13Q1-061:MOD13Q1.A2023273.h33v09.061.2023290142939","prev":null,"type":"FeatureCollection","context":{"limit":10,"returned":10},"features":[{"id":"MOD13Q1.A2023273.h35v10.061.2023290142559","bbox":[172.479315908761,-19.180676222654,180.143592885496,-9.97534112170732],"type":"Feature","links":[{"rel":"collection","href":"https://planetarycomputer.microsoft.com/api/stac/v1/collections/modis-13Q1-061","type":"application/json"},{"rel":"parent","href":"https://planetarycomputer.microsoft.com/api/stac/v1/collections/modis-13Q1-061","type":"application/json"},{"rel":"root","href":"https://planetarycomputer.microsoft.com/api/stac/v1","type":"application/json","title":"Microsoft Planetary Computer STAC API"},{"rel":"self","href":"https://planetarycomputer.microsoft.com/api/stac/v1/collections/modis-13Q1-061/items/MOD13Q1.A2023273.h35v10.061.2023290142559","type":"application/geo+json"},{"rel":"preview","href":"https://planetarycomputer.microsoft.com/api/data/v1/item/map?collecti

## Hydration
PgSTAC uses a method to compress the data stored by an Item by using the "item_assets" attribute on the Item's Collection. When an Item is added, a 'diff' is created between the Assets of an Item and the "item_assets" on the Collection. Any duplicate data in the Item content that can be derived from the Collection is then stripped from the Item content as it is stored in PgSTAC. PgSTAC refers to this process as 'dehydration'.

When a search is returned from PgSTAC, this information is then merged back into the Item pior to be returned. PgSTAC refersw to this process as 'hydration'.

For Collections that have very large Assets, this can save a huge amount of disk space as well as the availability of memory in the Postgres instance. This process is, however, more CPU intensive. There is an option when returning a search that allows for the return of the dehydrated data. This can be done using the '{"conf":{"nohydrate":true}}' parameter as part of the search.

Pypgstac has the ability to do shift this dehydration process away from the Database shifting the load to the application server and speeding things up, particularly under heavy load when multiple application server instances are able to process the results. At PgSTAC version 0.8, this process was sped up even further in pypgstac through implenting the dehydration in Rust. The default setting in STAC FastAPI is to use application side dehydration. 

In [77]:
%%psql -At
SET CLIENT_MIN_MESSAGES TO WARNING;
select search('{"limit":1}'::jsonb)->'features'->0->'assets'->'hdf';
select content->'assets'->'hdf' from items limit 1;
select content->'item_assets'->'hdf' from collections limit 1;


'SET'

{'href': 'https://modiseuwest.blob.core.windows.net/modis-061/MOD13Q1/35/10/2023273/MOD13Q1.A2023273.h35v10.061.2023290142559.hdf',
 'type': 'application/x-hdf',
 'roles': ['data'],
 'title': 'Source data containing all bands'}

{'href': 'https://modiseuwest.blob.core.windows.net/modis-061/MOD13Q1/35/10/2023241/MOD13Q1.A2023241.h35v10.061.2023258000322.hdf'}

{'type': 'application/x-hdf',
 'roles': ['data'],
 'title': 'Source data containing all bands'}

In [83]:
%%psql -At
select search()->'features'->0->'assets'->'hdf';


psql:/dev/stdin:1: NOTICE:  SEARCH:TOKEN: <NULL>
psql:/dev/stdin:1: NOTICE:  ***TOKEN: <NULL>
psql:/dev/stdin:1: NOTICE:  FULL WHERE CLAUSE:  TRUE 
psql:/dev/stdin:1: NOTICE:  Time to get counts and build query 4.645
psql:/dev/stdin:1: NOTICE:  Getting hydrated data.
psql:/dev/stdin:1: NOTICE:  CACHE SET TO f
psql:/dev/stdin:1: NOTICE:  Time to set hydration/formatting 0.748
psql:/dev/stdin:1: NOTICE:  Getting chunks for  TRUE  datetime DESC, id DESC
psql:/dev/stdin:1: NOTICE:  Running Query for 2023-01-01 00:00:00+00 to 2024-02-01 00:00:00+00. 3.428
psql:/dev/stdin:1: NOTICE:  Returned 11/11 Rows From 2023-01-01 00:00:00+00 to 2024-02-01 00:00:00+00. 0 to go. Time: 1.022ms
psql:/dev/stdin:1: NOTICE:  SEARCH_ROWS TOOK 4.779ms
psql:/dev/stdin:1: NOTICE:  Time to fetch rows 85.787
psql:/dev/stdin:1: NOTICE:  Query returned 11 records.
psql:/dev/stdin:1: NOTICE:  HAS NEXT | modis-13Q1-061:MOD13Q1.A2023273.h33v09.061.2023290142939
psql:/dev/stdin:1: NOTICE:  Time to get prev/next 0.949
psq

{'href': 'https://modiseuwest.blob.core.windows.net/modis-061/MOD13Q1/35/10/2023273/MOD13Q1.A2023273.h35v10.061.2023290142559.hdf',
 'type': 'application/x-hdf',
 'roles': ['data'],
 'title': 'Source data containing all bands'}

In [85]:
%%psql -At
select search('{"conf":{"nohydrate":true}}'::jsonb)->'features'->0->'assets'->'hdf';


psql:/dev/stdin:1: NOTICE:  SEARCH:TOKEN: <NULL>
psql:/dev/stdin:1: NOTICE:  ***TOKEN: <NULL>
psql:/dev/stdin:1: NOTICE:  FULL WHERE CLAUSE:  TRUE 
psql:/dev/stdin:1: NOTICE:  Time to get counts and build query 5.637
psql:/dev/stdin:1: NOTICE:  Getting non-hydrated data.
psql:/dev/stdin:1: NOTICE:  CACHE SET TO f
psql:/dev/stdin:1: NOTICE:  Time to set hydration/formatting 0.541
psql:/dev/stdin:1: NOTICE:  Getting chunks for  TRUE  datetime DESC, id DESC
psql:/dev/stdin:1: NOTICE:  Running Query for 2023-01-01 00:00:00+00 to 2024-02-01 00:00:00+00. 7.025
psql:/dev/stdin:1: NOTICE:  Returned 11/11 Rows From 2023-01-01 00:00:00+00 to 2024-02-01 00:00:00+00. 0 to go. Time: 3.812ms
psql:/dev/stdin:1: NOTICE:  SEARCH_ROWS TOOK 12.09ms
psql:/dev/stdin:1: NOTICE:  Time to fetch rows 48.106
psql:/dev/stdin:1: NOTICE:  Query returned 11 records.
psql:/dev/stdin:1: NOTICE:  HAS NEXT | modis-13Q1-061:MOD13Q1.A2023273.h33v09.061.2023290142939
psql:/dev/stdin:1: NOTICE:  Time to get prev/next 3.386

{'href': 'https://modiseuwest.blob.core.windows.net/modis-061/MOD13Q1/35/10/2023273/MOD13Q1.A2023273.h35v10.061.2023290142559.hdf'}